# Find UniProt IDs in ChEMBL targets

Ultimately, we are interested in getting [activity data from ChEMBl](/chembl-27/query_local_chembl-27.ipynb) we need to account for three components:

* The compound being measured
* The target the compound binds to
* The assay where this measurement took place

So, to find all activity data stored in ChEMBL that refers to kinases, we have to query for those assays annotated with a certain target.

Each of those three components have a unique ChEMBL ID, but so far we only have obtained Uniprot IDs in the `human-kinases` notebook. We need a way to connect Uniprot IDs to ChEMBL target IDs. Fortunately, ChEMBL maintains such a map in their FTP releases. We will parse that file and convert it into a dataframe for easy manipulation.

In [1]:
from pathlib import Path
import urllib.request

import pandas as pd

In [2]:
REPO = (Path(_dh[-1]) / "..").resolve()
DATA = REPO / 'data'

In [3]:
CHEMBL_VERSION = 28
url = fr"ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/releases/chembl_{CHEMBL_VERSION}/chembl_uniprot_mapping.txt"

In [4]:
with urllib.request.urlopen(url) as response:
    uniprot_map = pd.read_csv(response, sep="\t", skiprows=[0], names=["UniprotID", "chembl_targets", "description", "type"])
uniprot_map

,UniprotID,chembl_targets,description,type
0,P21266,CHEMBL2242,Glutathione S-transferase Mu 3,SINGLE PROTEIN
1,O00519,CHEMBL2243,Anandamide amidohydrolase,SINGLE PROTEIN
2,P19217,CHEMBL2244,Estrogen sulfotransferase,SINGLE PROTEIN
3,P97292,CHEMBL2245,Histamine H2 receptor,SINGLE PROTEIN
4,P17342,CHEMBL2247,Atrial natriuretic peptide receptor C,SINGLE PROTEIN
...,...,...,...,...
12930,Q91ZR5,CHEMBL3886121,Cation channel sperm-associated protein 1,SINGLE PROTEIN
12931,P48763,CHEMBL3886122,Sodium/hydrogen exchanger 2,SINGLE PROTEIN
12932,Q9UKU6,CHEMBL3886123,Thyrotropin-releasing hormone-degrading ectoen...,SINGLE PROTEIN
12933,Q9JJH7,CHEMBL3886124,Transient receptor potential cation channel su...,SINGLE PROTEIN


We join this new information to the human kinases aggregated list from `human-kinases` (all of them, regardless the source):

In [5]:
kinases = pd.read_csv(DATA / "human_kinases.aggregated.csv", index_col=0)
kinases

,UniprotID,Name,kinhub,klifs,pkinfam,reviewed_uniprot,dunbrack_msa
0,A0A0B4J2F2,SIK1B,False,False,True,True,True
1,A4QPH2,PI4KAP2,False,True,False,False,False
2,B5MCJ9,TRIM66,True,False,False,False,False
3,O00141,SGK1,True,True,True,True,True
4,O00238,BMPR1B|BMR1B,True,True,True,True,True
...,...,...,...,...,...,...,...
547,Q9Y616,IRAK3,True,True,True,True,True
548,Q9Y6E0,STK24,True,True,True,True,True
549,Q9Y6M4,KC1G3|CSNK1G3,True,True,True,True,True
550,Q9Y6R4,MAP3K4|M3K4,True,True,True,True,True


We are only interested in those kinases present in these datasets:

* KinHub
* KLIFS
* PKinFam
* Dunbrack's MSA

In [6]:
kinases_subset = kinases[kinases[["kinhub", "klifs", "pkinfam", "dunbrack_msa"]].sum(axis=1) > 0]
kinases_subset

,UniprotID,Name,kinhub,klifs,pkinfam,reviewed_uniprot,dunbrack_msa
0,A0A0B4J2F2,SIK1B,False,False,True,True,True
1,A4QPH2,PI4KAP2,False,True,False,False,False
2,B5MCJ9,TRIM66,True,False,False,False,False
3,O00141,SGK1,True,True,True,True,True
4,O00238,BMPR1B|BMR1B,True,True,True,True,True
...,...,...,...,...,...,...,...
547,Q9Y616,IRAK3,True,True,True,True,True
548,Q9Y6E0,STK24,True,True,True,True,True
549,Q9Y6M4,KC1G3|CSNK1G3,True,True,True,True,True
550,Q9Y6R4,MAP3K4|M3K4,True,True,True,True,True


We would also like to preserve the provenance of the Uniprot assignment, so we will group the provenance columns in a single one now.

In [7]:
kinases_subset["origin"] = kinases_subset.apply(lambda s: '|'.join([k for k in ["kinhub", "klifs", "pkinfam", "reviewed_uniprot", "dunbrack_msa"] if s[k]]), axis=1)
kinases_subset

,UniprotID,Name,kinhub,klifs,pkinfam,reviewed_uniprot,dunbrack_msa,origin
0,A0A0B4J2F2,SIK1B,False,False,True,True,True,pkinfam|reviewed_uniprot|dunbrack_msa
1,A4QPH2,PI4KAP2,False,True,False,False,False,klifs
2,B5MCJ9,TRIM66,True,False,False,False,False,kinhub
3,O00141,SGK1,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
4,O00238,BMPR1B|BMR1B,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
...,...,...,...,...,...,...,...,...
547,Q9Y616,IRAK3,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
548,Q9Y6E0,STK24,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
549,Q9Y6M4,KC1G3|CSNK1G3,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
550,Q9Y6R4,MAP3K4|M3K4,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


We can now merge the needed columns based on the `UniprotID` key.

In [8]:
merged = pd.merge(kinases_subset[["UniprotID", "Name", "origin"]], uniprot_map[["UniprotID", "chembl_targets", "type"]], how="inner", on='UniprotID')[["UniprotID", "Name", "chembl_targets", "type", "origin"]]
merged

,UniprotID,Name,chembl_targets,type,origin
0,A4QPH2,PI4KAP2,CHEMBL4105789,SINGLE PROTEIN,klifs
1,A4QPH2,PI4KAP2,CHEMBL3038509,PROTEIN COMPLEX,klifs
2,O00141,SGK1,CHEMBL2343,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
3,O00238,BMPR1B|BMR1B,CHEMBL5476,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
4,O00311,CDC7,CHEMBL2111377,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
...,...,...,...,...,...
964,Q9Y5S2,CDC42BPB|MRCKB,CHEMBL5052,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
965,Q9Y616,IRAK3,CHEMBL5081,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
966,Q9Y6E0,STK24,CHEMBL5082,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
967,Q9Y6M4,KC1G3|CSNK1G3,CHEMBL5084,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


How is this possible? 878 targets?!

Apparently, there's not 1:1 correspondence between UniprotID and ChEMBL ID! Some Uniprot IDs are included in several ChEMBL targets:

In [9]:
merged.UniprotID.value_counts()

P24941    15
P11802    15
P00519    11
P35968    10
O60885    10
          ..
Q96S44     1
P51617     1
Q09013     1
Q8IY84     1
O60285     1
Name: UniprotID, Length: 494, dtype: int64

In [10]:
merged[merged.UniprotID == "P11802"]

,UniprotID,Name,chembl_targets,type,origin
203,P11802,CDK4,CHEMBL331,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
204,P11802,CDK4,CHEMBL2095942,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
205,P11802,CDK4,CHEMBL2111326,SELECTIVITY GROUP,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
206,P11802,CDK4,CHEMBL1907601,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
207,P11802,CDK4,CHEMBL3301385,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
208,P11802,CDK4,CHEMBL4523686,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
209,P11802,CDK4,CHEMBL4523715,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
210,P11802,CDK4,CHEMBL4523732,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
211,P11802,CDK4,CHEMBL3038472,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
212,P11802,CDK4,CHEMBL3559691,PROTEIN FAMILY,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


... and some ChEMBL targets include several kinases (e.g. chimeric proteins):

In [11]:
merged[merged.chembl_targets == "CHEMBL2096618"]

,UniprotID,Name,chembl_targets,type,origin
105,P00519,ABL1,CHEMBL2096618,CHIMERIC PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
191,P11274,BCR,CHEMBL2096618,CHIMERIC PROTEIN,kinhub|klifs


This is due to the different `type` values:

In [12]:
merged.type.value_counts()

SINGLE PROTEIN                 491
PROTEIN FAMILY                 239
PROTEIN COMPLEX                118
PROTEIN-PROTEIN INTERACTION     81
SELECTIVITY GROUP               16
CHIMERIC PROTEIN                13
PROTEIN COMPLEX GROUP           11
Name: type, dtype: int64

If we focus on `SINGLE PROTEIN` types:

In [13]:
merged[merged.type == "SINGLE PROTEIN"]

,UniprotID,Name,chembl_targets,type,origin
0,A4QPH2,PI4KAP2,CHEMBL4105789,SINGLE PROTEIN,klifs
2,O00141,SGK1,CHEMBL2343,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
3,O00238,BMPR1B|BMR1B,CHEMBL5476,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
5,O00311,CDC7,CHEMBL5443,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
7,O00329,PIK3CD,CHEMBL3130,SINGLE PROTEIN,klifs|pkinfam
...,...,...,...,...,...
964,Q9Y5S2,CDC42BPB|MRCKB,CHEMBL5052,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
965,Q9Y616,IRAK3,CHEMBL5081,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
966,Q9Y6E0,STK24,CHEMBL5082,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
967,Q9Y6M4,KC1G3|CSNK1G3,CHEMBL5084,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


... we end up with a total of 485 targets, which is more acceptable.

For that reason, we will only save records corresponding to `type == SINGLE PROTEIN`

In [14]:
merged[merged.type == "SINGLE PROTEIN"].to_csv(DATA /  f"human_kinases_and_chembl_targets.chembl_{CHEMBL_VERSION}.csv", index=False)